# NLP Miniproject

### Comprehensive Answer Evaluation system
**- Bodhisatya Ghosh and Anish Gade**

In [ ]:
from functions import *

**Sytem inputs**

- Evaluation type
- Synpotic Answer
    - Subjective
    - Keyword based (',' separated keywords)
- Submitted answer
- Maximum marks

In [ ]:
evaluation = input("Enter evaluation type: ")
synoptic = input("Enter synoptic answer: ")
submitted = input("Enter submitted answer: ")
marks = int(input("Enter maximum marks: "))

In [ ]:
evaluate(evaluation, synoptic, submitted, marks)

### Functions used

In [ ]:
def evaluate(evaluation: str, synoptic: list[str] | str, submitted: str, marks: int):
    score_factor = None     
    
    if(evaluation == 'keyword'):
        score_factor = keyphrase_match_score(synoptic.split(','),submitted)
    
    elif(evaluation == 'subjective'):
        score_factor = get_similiarity(synoptic, submitted)

    return marks * score_factor

In [ ]:
def keyphrase_match_score(list_of_key_phrases: list[str], answer: str):
    nlp = spacy.load('en_core_web_sm')
    score = 0
    
    for phrase in list_of_key_phrases:
        phrase = re.sub(r'[^a-zA-z0-9 ]',' ',phrase)
        phrase = re.sub(r'\s+',' ',phrase)

        m_tool = Matcher(nlp.vocab)

        vocab = [{'LOWER':word} for word in phrase.split()]
        m_tool.add('matcher',[vocab,])
        sentence = nlp(answer)
        if len(m_tool(sentence)) > 0:
            score+=1


    return score/len(list_of_key_phrases)

* Takes all given synoptic keyphrases and uses Regular Expression to clean keyphrases
* Create a PhraseMatcher object from SpaCy
* Match phrases with submitted answer to check for number of phrases matched out of total needed phrases
* Returns ratio of number of phrases matched to total needed phrases

In [ ]:
def get_similiarity(synoptic, answer):
    sentences =[synoptic, answer]
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    embeddings = model.encode(sentences)
    similiarity = cosine_similarity(embeddings[0].reshape(1,-1),embeddings[1].reshape(1,-1))

    return similiarity[0][0]

* Takes all given synoptic model answer and submitted answer 
* Create a SentenceTransformer Object that uses 'paraphrase-multilingual-MiniLM-L12-v2'
* Encode sentences into feature vectors as trained by the model to acquire embeddings
* Returns cosine similiarity between model answer and submitted answers' feature vector embeddings as ametric of similiarity